<a href="https://colab.research.google.com/github/ojamoo/My-Git-Project/blob/master/StageC_Hamoye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn.utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import imblearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import  StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score,accuracy_score,precision_score,f1_score,confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import  StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.tree import DecisionTreeClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')

In [3]:
print(df)

          tau1      tau2      tau3  ...        g4      stab     stabf
0     2.959060  3.079885  8.381025  ...  0.958034  0.055347  unstable
1     9.304097  4.902524  3.047541  ...  0.781760 -0.005957    stable
2     8.971707  8.848428  3.046479  ...  0.109853  0.003471  unstable
3     0.716415  7.669600  4.486641  ...  0.362718  0.028871  unstable
4     3.134112  7.608772  4.943759  ...  0.820923  0.049860  unstable
...        ...       ...       ...  ...       ...       ...       ...
9995  2.930406  9.487627  2.376523  ...  0.608385  0.023892  unstable
9996  3.392299  1.274827  2.954947  ...  0.366120 -0.025803    stable
9997  2.364034  2.842030  8.776391  ...  0.145984 -0.031810    stable
9998  9.631511  3.994398  2.757071  ...  0.818391  0.037789  unstable
9999  6.530527  6.781790  4.349695  ...  0.942631  0.045263  unstable

[10000 rows x 14 columns]


In [4]:
df.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stab', 'stabf'],
      dtype='object')

In [5]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [6]:
#dependent variables
y=df[['stab','stabf']]

In [7]:
print(y)

          stab     stabf
0     0.055347  unstable
1    -0.005957    stable
2     0.003471  unstable
3     0.028871  unstable
4     0.049860  unstable
...        ...       ...
9995  0.023892  unstable
9996 -0.025803    stable
9997 -0.031810    stable
9998  0.037789  unstable
9999  0.045263  unstable

[10000 rows x 2 columns]


In [8]:
#predictive features
x=df.drop(columns=['stab','stabf'])

In [9]:
print(x)

          tau1      tau2      tau3  ...        g2        g3        g4
0     2.959060  3.079885  8.381025  ...  0.859578  0.887445  0.958034
1     9.304097  4.902524  3.047541  ...  0.862414  0.562139  0.781760
2     8.971707  8.848428  3.046479  ...  0.766689  0.839444  0.109853
3     0.716415  7.669600  4.486641  ...  0.976744  0.929381  0.362718
4     3.134112  7.608772  4.943759  ...  0.455450  0.656947  0.820923
...        ...       ...       ...  ...       ...       ...       ...
9995  2.930406  9.487627  2.376523  ...  0.779642  0.813512  0.608385
9996  3.392299  1.274827  2.954947  ...  0.567242  0.285880  0.366120
9997  2.364034  2.842030  8.776391  ...  0.986505  0.149286  0.145984
9998  9.631511  3.994398  2.757071  ...  0.587558  0.889118  0.818391
9999  6.530527  6.781790  4.349695  ...  0.505441  0.378761  0.942631

[10000 rows x 12 columns]


In [10]:
#using stabf (with a binary classification) as the sole dependent variable
y=y.drop(columns=['stab'])

In [11]:
print(y)

         stabf
0     unstable
1       stable
2     unstable
3     unstable
4     unstable
...        ...
9995  unstable
9996    stable
9997    stable
9998  unstable
9999  unstable

[10000 rows x 1 columns]


In [12]:
#checking distribution of target variable
y['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [13]:
#splitting the data into training and testing dataset
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [14]:
print(x_train)

          tau1      tau2      tau3  ...        g2        g3        g4
2694  6.255995  2.542401  7.024714  ...  0.618403  0.685739  0.660088
5140  5.070581  5.490253  8.075688  ...  0.097244  0.916955  0.129254
2568  1.220072  8.804028  3.874283  ...  0.923594  0.238881  0.660156
3671  7.498402  6.697603  8.798626  ...  0.899003  0.964752  0.600598
7427  7.074006  1.337511  6.100756  ...  0.716082  0.836928  0.165162
...        ...       ...       ...  ...       ...       ...       ...
2895  9.504984  5.265415  2.029617  ...  0.611297  0.080008  0.350312
7813  8.035818  4.632548  1.177810  ...  0.317770  0.726623  0.441620
905   7.052560  3.264057  1.405949  ...  0.605566  0.100591  0.747320
5192  5.085242  1.790031  2.486392  ...  0.418512  0.326253  0.982488
235   1.205354  6.995045  5.930860  ...  0.202768  0.848373  0.739993

[8000 rows x 12 columns]


In [15]:
print(y_train)

         stabf
2694  unstable
5140  unstable
2568  unstable
3671  unstable
7427  unstable
...        ...
2895    stable
7813    stable
905   unstable
5192  unstable
235     stable

[8000 rows x 1 columns]


In [16]:
print(x_test)

          tau1      tau2      tau3  ...        g2        g3        g4
9953  6.877876  4.113820  9.356768  ...  0.845536  0.112440  0.822562
3850  5.802841  6.271371  4.731540  ...  0.416478  0.912846  0.861306
4962  2.286998  4.385142  2.830232  ...  0.130186  0.703887  0.063811
3886  5.019920  2.209962  6.266080  ...  0.065992  0.427349  0.814648
5437  7.646145  9.187896  5.484219  ...  0.121611  0.787318  0.300314
...        ...       ...       ...  ...       ...       ...       ...
3919  8.320531  3.394352  1.468729  ...  0.493384  0.178907  0.173240
162   2.290102  3.028512  8.071234  ...  0.770194  0.234290  0.225070
7903  7.848945  0.684889  0.728223  ...  0.559799  0.682418  0.886902
2242  2.173967  5.776797  4.597638  ...  0.743103  0.152161  0.305183
2745  1.467618  9.389085  6.019740  ...  0.273217  0.526145  0.851163

[2000 rows x 12 columns]


In [17]:
print(y_test)

         stabf
9953  unstable
3850  unstable
4962    stable
3886    stable
5437  unstable
...        ...
3919    stable
162     stable
7903    stable
2242  unstable
2745  unstable

[2000 rows x 1 columns]


In [18]:
#scaling with standard scaler
scaler=StandardScaler()

In [19]:
scaled_train_df=scaler.fit_transform(x_train)
scaled_train_df=pd.DataFrame(scaled_train_df,columns=x_train.columns)

In [20]:
print(scaled_train_df)

          tau1      tau2      tau3  ...        g2        g3        g4
0     0.367327 -0.986042  0.650447  ...  0.339859  0.585568  0.492239
1    -0.064659  0.089437  1.035079  ... -1.558488  1.429649 -1.443521
2    -1.467850  1.298418 -0.502536  ...  1.451534 -1.045743  0.492489
3     0.820081  0.529920  1.299657  ...  1.361958  1.604140  0.275303
4     0.665424 -1.425627  0.312300  ...  0.695660  1.137504 -1.312575
...        ...       ...       ...  ...       ...       ...       ...
7995  1.551314  0.007408 -1.177640  ...  0.313976 -1.625728 -0.637401
7996  1.015925 -0.223483 -1.489381  ... -0.755209  0.734821 -0.304433
7997  0.657609 -0.722756 -1.405888  ...  0.293100 -1.550587  0.810344
7998 -0.059316 -1.260532 -1.010471  ... -0.388255 -0.726781  1.667916
7999 -1.473214  0.638438  0.250122  ... -1.174110  1.179282  0.783627

[8000 rows x 12 columns]


In [21]:
scaled_test_df=scaler.fit_transform(x_test)
scaled_test_df=pd.DataFrame(scaled_test_df,columns=x_test.columns)

In [22]:
print(scaled_test_df)

          tau1      tau2      tau3  ...        g2        g3        g4
0     0.592163 -0.420565  1.472472  ...  1.175287 -1.492644  1.086291
1     0.199183  0.364543 -0.190076  ... -0.395949  1.412703  1.227535
2    -1.086035 -0.321834 -0.873505  ... -1.444370  0.654216 -1.679799
3    -0.087014 -1.113357  0.361518  ... -1.679450 -0.349573  1.057439
4     0.873004  1.425833  0.080476  ... -1.475773  0.957057 -0.817608
...        ...       ...       ...  ...       ...       ...       ...
1995  1.119526 -0.682371 -1.362900  ... -0.114316 -1.251379 -1.280864
1996 -1.084900 -0.815496  1.010385  ...  0.899379 -1.050349 -1.091915
1997  0.947138 -1.668313 -1.629077  ...  0.128899  0.576286  1.320848
1998 -1.127354  0.184573 -0.238207  ...  0.800172 -1.348463 -0.799858
1999 -1.385560  1.499043  0.272971  ... -0.920581  0.009039  1.190559

[2000 rows x 12 columns]


In [23]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

In [24]:
r_f_c=RandomForestClassifier(n_estimators=100)
r_f_c.fit(scaled_train_df,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [25]:
y_predict_rfc=r_f_c.predict(scaled_test_df)

In [26]:
print(y_predict_rfc)

['unstable' 'unstable' 'stable' ... 'stable' 'unstable' 'unstable']


In [27]:
#Confusion Matrix for Random Forest Classifier
cnf_mat=confusion_matrix(y_true=y_test,y_pred=y_predict_rfc,labels=['stable','unstable'])

In [28]:
print(cnf_mat)

[[ 621   91]
 [  64 1224]]


In [29]:
#Accuracy for Random Forest Classifier
accuracy=accuracy_score(y_test,y_predict_rfc)
print('Accuracy: {}'.format(round(accuracy,4)))

Accuracy: 0.9225


In [30]:
#Extra-Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier

In [31]:
e_t_c=ExtraTreesClassifier(random_state=1)
e_t_c.fit(scaled_train_df,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [32]:
y_predict_etc=e_t_c.predict(scaled_test_df)

In [33]:
print(y_predict_etc)

['unstable' 'unstable' 'stable' ... 'stable' 'unstable' 'unstable']


In [34]:
#Confusion Matrix for Extra-Trees Classifier
cnf_mat_etc=confusion_matrix(y_true=y_test,y_pred=y_predict_etc,labels=['stable','unstable'])

In [35]:
print(cnf_mat_etc)

[[ 602  110]
 [  38 1250]]


In [36]:
#Accuracy for Extra-Trees Classifier
accuracy=accuracy_score(y_test,y_predict_etc)
print('Accuracy: {}'.format(round(accuracy,4)))

Accuracy: 0.926


In [37]:
#XGBoost
from xgboost import XGBClassifier

In [38]:
xgb=XGBClassifier(random_state=1)

In [39]:
xgb.fit(scaled_train_df,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [40]:
y_predict_xgb=xgb.predict(scaled_test_df)

In [41]:
print(y_predict_xgb)

['unstable' 'unstable' 'stable' ... 'stable' 'unstable' 'unstable']


In [42]:
#Confusion Matrix for XGBoost Classifier
cnf_mat_xgb=confusion_matrix(y_true=y_test,y_pred=y_predict_xgb,labels=['stable','unstable'])

In [43]:
print(cnf_mat_xgb)

[[ 599  113]
 [  49 1239]]


In [44]:
#Accuracy for XGBoost Classifier
accuracy=accuracy_score(y_test,y_predict_xgb)
print('Accuracy: {}'.format(round(accuracy,4)))

Accuracy: 0.919


In [45]:
#Light Gradient Boosting Classifier
import lightgbm as lgb

In [46]:
lgb_c= lgb.LGBMClassifier(random_state=1)
lgb_c.fit(scaled_train_df,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [47]:
y_predict_lgb = lgb_c.predict(scaled_test_df)

In [48]:
print(y_predict_lgb)

['unstable' 'unstable' 'stable' ... 'stable' 'unstable' 'unstable']


In [49]:
#Confusion Matrix for Light Gradient Boosting Classifier
cnf_mat_lgb=confusion_matrix(y_true=y_test,y_pred=y_predict_lgb,labels=['stable','unstable'])

In [50]:
print(cnf_mat_lgb)

[[ 635   77]
 [  52 1236]]


In [51]:
#Accuracy for Light Gradient Boosting Classifier
accuracy=accuracy_score(y_test,y_predict_lgb)
print('Accuracy: {}'.format(round(accuracy,4)))

Accuracy: 0.9355


In [52]:
#Parameters
n_estimators=[50,100,300,500,1000] 
min_samples_split=[2,3,5,7,9] 
min_samples_leaf=[1,2,4,6,8] 
max_features=['auto','sqrt','log2',None] 
hyperparameter_grid={'n_estimators':n_estimators,'min_samples_leaf':min_samples_leaf,'min_samples_split':min_samples_split,'max_features':max_features}

In [53]:
#Randomized Search CV
from sklearn.model_selection import RandomizedSearchCV

In [54]:
random_search_cv=RandomizedSearchCV(estimator=e_t_c,param_distributions=hyperparameter_grid,random_state=1)
random_search_cv=random_search_cv.fit(scaled_train_df,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

In [55]:
random_search_cv.best_params_

{'max_features': None,
 'min_samples_leaf': 8,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [56]:
random_search_cv.best_estimator_

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features=None,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=8, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [57]:
#new Extra-Trees Classifier
new_etc=ExtraTreesClassifier(random_state=1,n_estimators=1000,min_samples_leaf=8,max_features=None,min_samples_split=2)
new_etc.fit(scaled_train_df,y_train)
new_predict_etc=new_etc.predict(scaled_test_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
accuracy=accuracy_score(y_test,new_predict_etc)
print('Accuracy: {}'.format(round(accuracy,4)))

Accuracy: 0.9285


In [59]:
#feature importance
feat_imp=random_search_cv.best_estimator_.feature_importances_

In [60]:
feat_imp

array([0.13723975, 0.1405075 , 0.13468029, 0.13541676, 0.00368342,
       0.00533686, 0.00542927, 0.00496249, 0.10256244, 0.10757765,
       0.11306268, 0.10954089])

In [61]:
sorted(zip(feat_imp,x))

[(0.003683422151688322, 'p1'),
 (0.004962486591192238, 'p4'),
 (0.005336864710946151, 'p2'),
 (0.005429268421191957, 'p3'),
 (0.10256244080927947, 'g1'),
 (0.10757764577478764, 'g2'),
 (0.10954089174337298, 'g4'),
 (0.11306267999167334, 'g3'),
 (0.13468028520386593, 'tau3'),
 (0.1354167630909727, 'tau4'),
 (0.13723974766109256, 'tau1'),
 (0.14050750384993677, 'tau2')]